### CREATION DE LA BD

In [15]:
import sqlite3

from debugpy.adapter.servers import connections
from groq import BaseModel
from streamlit import cursor

In [16]:
import pandas as pd

In [17]:
connection = sqlite3.connect("mydb.db")


In [18]:
connection

In [19]:
table_creation_query="""
CREATE TABLE IF NOT EXISTS employees (
    emp_id INTEGER PRIMARY KEY ,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT unique NOT NULL,
    hire_date TEXT NOT NULL,
    salary REAL NOT NULL
);
"""


In [20]:
table_creation_query2="""
CREATE TABLE IF NOT EXISTS customers (
    customer_id INTEGER PRIMARY KEY ,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT unique NOT NULL,
    phone TEXT
);
"""

In [21]:
table_creation_query3="""
CREATE TABLE IF NOT EXISTS orders (
    order_id INTEGER PRIMARY KEY ,
    customer_id INTEGER NOT NULL,
    order_date TEXT NOT NULL,
    amount REAL NOT NULL,
    FOREIGN KEY(customer_id) REFERENCES customers (customer_id)
);
"""

In [22]:
cusor = connection.cursor()

In [23]:
cusor.execute(table_creation_query)
cusor.execute(table_creation_query2)
cusor.execute(table_creation_query3)

In [24]:
insert_query = """
INSERT INTO employees(emp_id, first_name, last_name, email, hire_date, salary) values (?, ?, ?, ?, ?, ?); """

insert_query_customers = """
INSERT INTO customers(customer_id, first_name, last_name, email, phone) values (?, ?, ?, ?, ?); """

insert_query_orders = """
INSERT INTO orders(order_id, customer_id, order_date, amount) values (?, ?, ?, ?); """  # Correction: order_date

employee_data = [
    (1, 'John', 'Doe', 'john.doe@example.com', '2022-05-15', 55000),
    (2, 'Jane', 'Smith', 'jane.smith@example.com', '2021-03-10', 60000),
    (3, 'Michael', 'Johnson', 'michael.johnson@example.com', '2020-07-22', 65000),
]

customer_data = [
    (1, 'Alice', 'Brown', 'alice.brown@example.com', '123-456-7890'),
    (2, 'Bob', 'Davis', 'bob.davis@example.com', '987-654-3210'),
    (3, 'Charlie', 'Wilson', 'charlie.wilson@example.com', '555-123-4567'),
]

order_data = [
    (1, 1, '2023-01-10', 200.75),
    (2, 2, '2022-11-15', 150.50),
    (3, 3, '2023-02-01', 300.00),
]

# Exécution des requêtes d'insertion
cusor.executemany(insert_query, employee_data)
cusor.executemany(insert_query_customers, customer_data)
cusor.executemany(insert_query_orders, order_data)


IntegrityError: UNIQUE constraint failed: employees.emp_id

In [12]:
connection.commit()

In [27]:
cusor.execute('SELECT * FROM customers;')

In [28]:
for row in cusor.fetchall():
    print(row)

(1, 'Alice', 'Brown', 'alice.brown@example.com', '123-456-7890')
(2, 'Bob', 'Davis', 'bob.davis@example.com', '987-654-3210')
(3, 'Charlie', 'Wilson', 'charlie.wilson@example.com', '555-123-4567')


In [33]:
from langchain_community.utilities import SQLDatabase

In [37]:
#from langchain_community.utilities import SQLDatabase

# Inclure sqlite:///
db = SQLDatabase.from_uri("sqlite:///mydb.db")


In [38]:
db

In [39]:
db.dialect

'sqlite'

In [41]:
db.get_usable_table_names()

['customers', 'employees', 'orders']

In [3]:
from langchain_groq import ChatGroq

# Votre clé API
api_key = "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"

# Initialiser le client ChatGroq avec la clé API
llm = ChatGroq(model="llama3-8b-8192", api_key=api_key)

# Exemple d'appel au modèle



In [4]:
response = llm.invoke("Hello, how are you?")
print(response)

content="I'm just an AI, I don't have feelings like humans do, but I'm functioning properly and ready to assist you with any questions or tasks you may have! It's great to chat with you. How can I help you today?" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 16, 'total_tokens': 66, 'completion_time': 0.041666667, 'prompt_time': 0.005745321, 'queue_time': 0.572284951, 'total_time': 0.047411988}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None} id='run-67c675e5-30ad-4b20-af8d-4b6c7ffd68a7-0' usage_metadata={'input_tokens': 16, 'output_tokens': 50, 'total_tokens': 66}


In [68]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

In [69]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [72]:
tools=toolkit.get_tools()

In [73]:
for tool in tools:
    print(tool.name)

sql_db_query
sql_db_schema
sql_db_list_tables
sql_db_query_checker


In [75]:
list_tables_tool = next((tool for tool in tools if tool.name == "sql_db_list_tables"),None)

In [76]:
list_tables_tool

ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000266858389D0>)

In [77]:
get_schema_tool = next((tool for tool in tools if tool.name == "sql_db_schema"),None)

In [78]:
get_schema_tool

InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000266858389D0>)

In [80]:
list_tables_tool.invoke("")

'customers, employees, orders'

In [83]:
print(get_schema_tool.invoke("customers"))


CREATE TABLE customers (
	customer_id INTEGER, 
	first_name TEXT NOT NULL, 
	last_name TEXT NOT NULL, 
	email TEXT NOT NULL, 
	phone TEXT, 
	PRIMARY KEY (customer_id), 
	UNIQUE (email)
)

/*
3 rows from customers table:
customer_id	first_name	last_name	email	phone
1	Alice	Brown	alice.brown@example.com	123-456-7890
2	Bob	Davis	bob.davis@example.com	987-654-3210
3	Charlie	Wilson	charlie.wilson@example.com	555-123-4567
*/


In [88]:
from langchain_core.tools import tool

@tool
def db_query_tool(query: str) -> str:
    """
    Executes a database query and returns the result.

    Parameters:
        query (str): The database query string to be executed.

    Returns:
        str: The result of the query, or an error message if the query fails.
    """
    result = db.run_no_throw(query)
    if not result:
        return "Error: Query failed. Rewrite your query."
    return result


In [89]:
db.run("select * from customers")

"[(1, 'Alice', 'Brown', 'alice.brown@example.com', '123-456-7890'), (2, 'Bob', 'Davis', 'bob.davis@example.com', '987-654-3210'), (3, 'Charlie', 'Wilson', 'charlie.wilson@example.com', '555-123-4567')]"

In [94]:
db_query_tool.invoke("select * from customers")

"[(1, 'Alice', 'Brown', 'alice.brown@example.com', '123-456-7890'), (2, 'Bob', 'Davis', 'bob.davis@example.com', '987-654-3210'), (3, 'Charlie', 'Wilson', 'charlie.wilson@example.com', '555-123-4567')]"

## query_check

## query_gen

In [117]:
import warnings
warnings.filterwarnings("ignore")

In [116]:
from typing import Annotated, Literal
from langchain_core.messages import AIMessage
from langchain_core.pydantic_v1 import BaseModel, Field # Updated to import directly from pydantic
from typing_extensions import TypedDict
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import AnyMessage, add_messages
from typing import Any
from langchain_core.runnables import RunnableLambda, RunnableWithFallbacks
from langgraph.prebuilt import ToolNode

In [102]:
from langchain_core.prompts import ChatPromptTemplate

In [118]:
from langchain_core.prompts import ChatPromptTemplate

query_check_system = """You are a SQL expert with a strong attention to detail.
Double check the SQLite query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

If there are any of the above mistakes, rewrite the query. If there are no mistakes, just reproduce the original query.

You will call the appropriate tool to execute the query after running this check."""

query_check_prompt = ChatPromptTemplate.from_messages([("system", query_check_system), ("placeholder", "{messages}")])

query_check = query_check_prompt | llm.bind_tools([db_query_tool])

In [119]:


query_check.invoke({"messages": [("user", "SELECT * FROM Employees LIMIT 5;")]})



AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_90t8', 'function': {'arguments': '{"query":"SELECT * FROM Employees LIMIT 5"}', 'name': 'db_query_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 1083, 'total_tokens': 1156, 'completion_time': 0.060833333, 'prompt_time': 0.131706685, 'queue_time': 0.020172300000000004, 'total_time': 0.192540018}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3c0195db-89fb-4ba3-9da7-db1eecceea4a-0', tool_calls=[{'name': 'db_query_tool', 'args': {'query': 'SELECT * FROM Employees LIMIT 5'}, 'id': 'call_90t8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1083, 'output_tokens': 73, 'total_tokens': 1156})

In [120]:
class SubmitFinalAnswer(BaseModel):
    """Submit the final answer to the user based on the query results."""
    final_answer: str = Field(..., description="The final answer to the user")


# Add a node for a model to generate a query based on the question and schema
query_gen_system = """You are a SQL expert with a strong attention to detail.

Given an input question, output a syntactically correct SQLite query to run, then look at the results of the query and return the answer.

DO NOT call any tool besides SubmitFinalAnswer to submit the final answer.

When generating the query:

Output the SQL query that answers the input question without a tool call.

Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.

If you get an error while executing a query, rewrite the query and try again.

If you get an empty result set, you should try to rewrite the query to get a non-empty result set.
NEVER make stuff up if you don't have enough information to answer the query... just say you don't have enough information.

If you have enough information to answer the input question, simply invoke the appropriate tool to submit the final answer to the user.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database. Do not return any sql query except answer."""


query_gen_prompt = ChatPromptTemplate.from_messages([("system", query_gen_system), ("placeholder", "{messages}")])

query_gen = query_gen_prompt | llm.bind_tools([SubmitFinalAnswer])

In [121]:
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [122]:
def first_tool_call(state:State)->dict[str,list[AIMessage]]:
    return{"messages": [AIMessage(content="",tool_calls=[{"name":"sql_db_list_tables","args":{},"id":"tool_abcd123"}])]}

In [123]:
def handle_tool_error(state:State):
    error = state.get("error")
    tool_calls = state["messages"][-1].tool_calls
    return {
        "messages": [
        ToolMessage(content=f"Error: {repr(error)}\n please fix your mistakes.",tool_call_id=tc["id"],)
        for tc in tool_calls
        ]
    }

In [124]:
def create_tool_node_with_fallback(tools:list)-> RunnableWithFallbacks[Any, dict]:
    return ToolNode(tools).with_fallbacks([RunnableLambda(handle_tool_error)], exception_key="error")


In [125]:
def model_check_query(state:State):
    return {"messages": [query_check.invoke({"messages": [state["messages"][-1]]})]}

In [126]:
def query_gen_node(state:State):
    message = query_gen.invoke(state)

    # Sometimes, the LLM will hallucinate and call the wrong tool. We need to catch this and return an error message.
    tool_messages = []
    if message.tool_calls:
        for tc in message.tool_calls:
            if tc["name"] != "SubmitFinalAnswer":
                tool_messages.append(
                    ToolMessage(
                        content=f"Error: The wrong tool was called: {tc['name']}. Please fix your mistakes. Remember to only call SubmitFinalAnswer to submit the final answer. Generated queries should be outputted WITHOUT a tool call.",
                        tool_call_id=tc["id"],
                    )
                )
    else:
        tool_messages = []
    return {"messages": [message] + tool_messages}

In [127]:
def should_continue(state:State):
    messages = state["messages"]
    last_message = messages[-1]
    if getattr(last_message, "tool_calls", None):
        return END
    if last_message.content.startswith("Error:"):
        return "query_gen"
    else:
        return "correct_query"

In [128]:
workflow = StateGraph(State)

In [129]:
workflow.add_node("first_tool_call",first_tool_call)
workflow.add_node("list_tables_tool", create_tool_node_with_fallback([list_tables_tool]))
workflow.add_node("get_schema_tool", create_tool_node_with_fallback([get_schema_tool]))
model_get_schema = llm.bind_tools([get_schema_tool])
workflow.add_node("model_get_schema",lambda state: {"messages": [model_get_schema.invoke(state["messages"])],},)
workflow.add_node("query_gen", query_gen_node)
workflow.add_node("correct_query", model_check_query)
workflow.add_node("execute_query", create_tool_node_with_fallback([db_query_tool]))

In [130]:
workflow.add_edge(START, "first_tool_call")
workflow.add_edge("first_tool_call", "list_tables_tool")
workflow.add_edge("list_tables_tool", "model_get_schema")
workflow.add_edge("model_get_schema", "get_schema_tool")
workflow.add_edge("get_schema_tool", "query_gen")
workflow.add_conditional_edges("query_gen",should_continue,
                               {END:END,
                                "query_gen":"query_gen",
                                "correct_query":"correct_query"})
workflow.add_edge("correct_query", "execute_query")
workflow.add_edge("execute_query", "query_gen")

In [131]:
app=workflow.compile()

In [132]:
query={"messages": [("user", "how many order are there which is more than 300 rupees?")]}

In [133]:
response=app.invoke(query)

In [134]:
response["messages"][-1].tool_calls[0]["args"]["final_answer"]

'There are 1 orders which are more than 300 rupees'

In [136]:
query = {"messages": [("user", "how many orders are there which are more than 300 rupees?")]}
app.stream(input=query)

<generator object Pregel.stream at 0x000002668B2DC610>